In [ ]:
# The script used here can be found in diHMM website
https://github.com/gcyuan/diHMM-cpp


# Set dir

In [ ]:
work_dir = "diHMM_output"


# Step1: create binarized file

In [ ]:
script_sh="bdg_to_binarize.sh"
chrom_size="mm10.chrom.sizes.txt"
bin_size=100
input_bdg_dir="bdg/"
out_dir="1.binary/"
clusters=("C1" "C2" "C3" "C4" "C5" "C6" "C7" "C8" "C9")
mark="H3K4me3"
cutoff=5

mkdir 1.binary
num=${#clusters[@]}
for ((i=0;i<$num;i++))
do
        cluster=${clusters[i]}
        bdg=${input_bdg_dir}/${cluster}.bdg
        sh $script_sh -i $bdg -o $out_dir -m $mark -t $cluster -s $bin_size -c $chrom_size -f $cutoff
done

In [ ]:
out_dir=1.binary_bed
mkdir $out_dir

bedf=${out_dir}/mm10_100bp.bed
chrom_size="mm10.chrom.sizes.txt"
bedtools makewindows -g $chrom_size -w 100 > $bedf

mark=H3K4me3
clusters=("C1" "C2" "C3" "C4" "C5" "C6" "C7" "C8" "C9")
d=1.binary/
for c in ${clusters[@]}
do
        o=$out_dir/${c}_binary.bed
        f="$d/${c}_chr1_binary.txt ${d}/${c}_chr10_binary.txt ${d}/${c}_chr11_binary.txt \
        ${d}/${c}_chr12_binary.txt ${d}/${c}_chr13_binary.txt ${d}/${c}_chr14_binary.txt \
        ${d}/${c}_chr15_binary.txt ${d}/${c}_chr16_binary.txt ${d}/${c}_chr17_binary.txt \
        ${d}/${c}_chr18_binary.txt ${d}/${c}_chr19_binary.txt ${d}/${c}_chr2_binary.txt \
        ${d}/${c}_chr3_binary.txt ${d}/${c}_chr4_binary.txt ${d}/${c}_chr5_binary.txt \
        ${d}/${c}_chr6_binary.txt ${d}/${c}_chr7_binary.txt ${d}/${c}_chr8_binary.txt \
        ${d}/${c}_chr9_binary.txt ${d}/${c}_chrX_binary.txt"
        cat $f|grep -v $mark|grep -v chr|paste $bedf -|\
        perl -alne 'print if $F[3] == 1' > $o &
done
wait

# Step 2: running diHMM

## 2.1 running diHMM

In [ ]:
out_dir="2.diHMM_out"
mkdir $out_dir
cd $out_dir

mark="H3K4me3"
domain_size=5
domain_states=2
bin_states=2
bin_size=100

binary_dir=../1.binary/
log_dir="0.logs"
mkdir -p $log_dir

sc_py=dihmm-cpp/Train_diHMM.py
clusters=("C1" "C2" "C3" "C4" "C5" "C6" "C7" "C8" "C9")
chrs="chr1,chr2,chr3,chr4,chr5,chr6,chr7,chr8,chr9,chr10,chr11,chr12,chr13,chr14,chr15,chr16,chr17,chr18,chr19,chrX"
#chrs=(chr1 chr10 chr11 chr12 chr13 chr14 chr15 chr16 chr17 chr18 chr19 chr2 chr3 chr4 chr5 chr6 chr7 chr8 chr9 chrX)
for x in ${clusters[@]}
do
        o=$log_dir/$x.log;od=${x};
        bsub -P acc_YuanLab -W 4000  -R rusage[mem=50000] -n 1 -R span[hosts=1] -o $o \
        python $sc_py -i $binary_dir -o $od --clusters $x --chroms $chrs --n_bin_states $bin_states --n_domain_states $domain_states --domain_size $domain_size --bin_res $bin_size
done

## 2.2 get domain file

In [ ]:
in_dir="2.diHMM_out"
out_dir="2.domain_file"
mkdir -p $out_dir

clusters=("C1" "C2" "C3" "C4" "C5" "C6" "C7" "C8" "C9")
for x in ${clusters[@]}
do
        odomain=${out_dir}/domain_states_${x}.bed
        obin=${out_dir}/bin_states_${x}.bed

        cat $in_dir/${x}/anno/*_domain_states.bed |grep -v track|perl -alne 'if ($F[3] eq "N0"){$s="N1"};if ($F[3]eq "N1"){$s="N0"};$"="\t";print "@F[0..2]\t$s\t@F[4..$#F]"' > $odomain
        cat $in_dir/${x}/anno/*_bin_states.bed |grep -v track > $obin

done

## 2.3 get domain bed file 

In [ ]:
in_dir="2.domain_file/"
out_dir="3.domain_bed"
label=domain
mkdir $out_dir

clusters=("C1" "C2" "C3" "C4" "C5" "C6" "C7" "C8" "C9")
for x in ${clusters[@]}
do
        n=${label}"_states_"$x".bed"
        f=${in_dir}/$n
        o=${out_dir}/$n
        perl -alne '$"="\t";print "@F[0..2]" if $F[3] eq "N1"' $f|sort -k1,1 -k2,2n|perl -slane 'print "$_\t${l}_$."' -- -l=$label > $o &
done
wait

## 2.4 get bin bed file

In [ ]:
in_dir="2.domain_file/"
out_dir="3.bin_bed"
label=bin
mkdir $out_dir

clusters=("C1" "C2" "C3" "C4" "C5" "C6" "C7" "C8" "C9")
for x in ${clusters[@]}
do
        n=${label}"_states_"$x".bed"
        f=${in_dir}/$n
        o=${out_dir}/$n
        perl -alne '$"="\t";print "@F[0..2]" if $F[3] eq "N1"' $f|sort -k1,1 -k2,2n|perl -slane 'print "$_\t${l}_$."' -- -l=$label > $o &
done
wait